In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook as tqdm

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from torchtext.data import Field, TabularDataset, BucketIterator, Iterator
import torch.nn as nn
import torch.nn.functional as F
import transformers
from transformers import BertForSequenceClassification, BertTokenizerFast, AdamW
from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config, GPT2LMHeadModel

import torch.optim as optim

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import plotly.express as px
import seaborn as sns

import time
import warnings
import random
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
def remove_over(df, leng = 120, target='reviewText', labels='label'):
    '''Checks text length for being over 512 tokens, 
    which bert-base does not support.'''
    seq_list = []
    seq_len = [len(i.split()) for i in df[target]]
    ugh = []
    for x, y in enumerate(seq_len):
        if y > leng:
            ugh.append(x)
    
    df_tuple = zip(df[target], df[labels])
    
    for x, text in enumerate(df_tuple):
        if x not in ugh:
            seq_list.append(text)
            
    seq_df = pd.DataFrame(seq_list, columns=['reviews','labels'])
    
    return seq_df

In [3]:
class BERT_Arch(nn.Module):
    
    def __init__(self, bert):
        '''
        Professional docstring here
        '''
        
        super(BERT_Arch, self).__init__()
        self.bert = bert
        
        self.droput = nn.Dropout(0.1)
        
        self.relu = nn.ReLU()
        
        self.lstm = nn.LSTM(768,512, bidirectional=True, num_layers=2, dropout=.1)
        self.bn = nn.BatchNorm1d(1024)
        self.fc1 = nn.Linear(1024,512)
        self.bn1 = nn.BatchNorm1d(512)
        self.fc2 = nn.Linear(512,256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256,128)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128,2)
        
        self.softmax = nn.Softmax(dim=1)
        

    def forward(self, sent_id):
        '''
        Forward pass for model
        '''
        _, x = self.bert(sent_id)
        
        x = x.unsqueeze(0)
        x, _ = self.lstm(x)
        x = x.squeeze(0)
        x = self.bn(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.bn1(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.bn2(x)
        x = self.fc3(x)
        x = self.relu(x)
        x = self.bn3(x)
        x = self.fc4(x)
        
        x = self.softmax(x)
        
        return x

In [4]:
path = 'model_save/'

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

In [6]:
gpt_configuration = GPT2Config.from_pretrained(f'{path}config.json', output_hidden_states=False)
gpt_model = GPT2LMHeadModel.from_pretrained(f"{path}pytorch_model.bin", config=gpt_configuration)

In [7]:
gpt_tokenizer = GPT2Tokenizer.from_pretrained(pretrained_model_name_or_path=f'{path}',
                                          vocab_file=f'{path}vocab.json', merges_file=f'{path}merges.txt',
                                          bos_token='<|sot|>', eos_token='<|eot|>', pad_token='<|pad|>')

In [8]:
gpt_tokenizer

PreTrainedTokenizer(name_or_path='model_save/', vocab_size=50257, model_max_len=1024, is_fast=False, padding_side='right', special_tokens={'bos_token': AddedToken("<|sot|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'eos_token': AddedToken("<|eot|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'unk_token': AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True), 'pad_token': '<|pad|>'})

In [9]:
gpt_model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50260, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [10]:
bert_model = BERT_Arch('bert_base_uncased')
bert_model = torch.load('bert_model_save/pytorch_model.bin')

In [11]:
bert_model.eval()

BERT_Arch(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

In [12]:
bert_tokenizer = BertTokenizerFast.from_pretrained('bert_model_save/')

In [13]:
# class DetectionGAN(object):
#     def __init__(self, dataframe,
#                  gpt_model=gpt_model, gpt_tokenizer=gpt_tokenizer, BERT_model=bert_model, BERT_tokenizer=bert_tokenizer,
#                  num_labels=2):
        
#         self.dataframe = dataframe
#         print(self.dataframe)

#         self.device = torch.device('cpu')

#         self.bert_tokenizer = BERT_tokenizer
#         self.discriminator = BERT_model.to(self.device)
#         self.bert_optimizer = AdamW(self.discriminator.parameters(), lr=1e-5)

#         self.gpt2_tokenizer = gpt_tokenizer
#         self.generator = gpt_model.to(device)
#         self.gpt2_optimizer = AdamW(self.generator.parameters(), lr=1e-5)
        
#         torch.cuda.empty_cache()

#     def textGeneration(self, generator_input):
#         text_id = generator_input
#         input, past = torch.tensor([text_id]).to(self.device), None
#         for _ in range(random.randint(30, 100)):
#             logits, past = self.generator(input, past = past)
#             input = torch.multinomial(F.softmax(logits[:, -1]), 1)
#             text_id.append(input.item())
#         return self.gpt2_tokenizer.decode(text_id)
    
#     def dataGenerator(self, batch_size = 2):
#         # Randomly fetch traning data bunch
#         sample_text_ss = self.dataframe['reviews'].iloc[random.sample(range(len(self.dataframe)), batch_size)]
        
#         # Tokenize training data bunch with GPT2 tokenizer and take top 10 words
#         sample_text_encode_top10 = sample_text_ss.map(lambda x : self.gpt2_tokenizer.encode(x)[:10])
        
#         # Generate text using GPT2 generator
#         sample_text_generate_ss = sample_text_encode_top10.map(self.textGeneration)
#         return sample_text_generate_ss, sample_text_ss
    
#     def discriminatorInput(self, text):
#         input_token = ['[CLS]'] + self.bert_tokenizer.tokenize(text) + ['[SEP]']
#         input_id = self.bert_tokenizer.convert_tokens_to_ids(input_token)
#         return [input_id]
    
#     def saveGeneratedReview(self):
#         content = self.dataframe['reviews'].values[random.randint(0, len(self.dataframe))]
#         content_id = self.gpt2_tokenizer.encode(content)[:10]
#         return self.textGeneration(content_id), content
        
#     def train(self, num_epochs = 10, save_interval = 1):
#         start = time.time()
#         generated_review_list = []
#         real_review_list = []
#         d_loss_list = []
#         g_loss_list = []

#         for epoch in range(num_epochs):
#             print('-' * 10)
#             print('Epoch {}/{}'.format(epoch + 1, num_epochs))
#             print('-' * 10)

#             # Load in data
#             sample_text_generate_ss, sample_text_ss = self.dataGenerator(batch_size = 2)

#             # Convert generated text and real text bunch to WorkPiece encode ID as discriminator input
#             discriminator_input_ss = pd.concat([sample_text_generate_ss, sample_text_ss], axis = 0, ignore_index = True).map(self.discriminatorInput)
#             discriminator_input = torch.LongTensor(np.array(pd.DataFrame(discriminator_input_ss.sum()).fillna(0).astype('int32'))).to(self.device)
#             discriminator_input_generate = discriminator_input[:len(sample_text_generate_ss)].to(self.device)


#             # Create labels for training discriminator and generator
#             labels = torch.LongTensor([0] * len(sample_text_generate_ss) + [1] * len(sample_text_ss)).to(self.device)
#             valid = torch.LongTensor([1] * len(sample_text_ss)).to(self.device)

#             # Each epoch has a train_discriminator and train_generator phase
#             for phase in ['train_discriminator', 'train_generator']:
#                 if phase == 'train_discriminator':
#                     # Set discriminator to training mode
#                     print(phase)
#                     self.discriminator.train()
#                     # Freeze all trainable parameters
#                     for param in self.discriminator.parameters():
#                         param.requires_grad = True
#                     # Zero the discriminator parameter gradients
#                     self.bert_optimizer.zero_grad()
#                     # Forward propagation
#                     d_loss = self.discriminator(sent_id = discriminator_input).mean()
#                     # Backward propagation
#                     d_loss.backward()
#                     self.bert_optimizer.step()

#                 else:
#                     print(phase)
#                     # Set discriminator to evaluate mode
#                     self.discriminator.eval()

#                     # Zero the generator parameter gradients
#                     self.gpt2_optimizer.zero_grad()

#                     # Forward propagation
#                     g_loss = self.discriminator(sent_id = discriminator_input_generate).mean()

#                     # Backward propagation
#                     g_loss.backward()
#                     self.gpt2_optimizer.step()   

#             # Plot the progress
#             print('Discriminator Loss:', d_loss.item())
#             print('Generator Loss:', g_loss.item())
#             print()
#             d_loss_list.append(d_loss)
#             g_loss_list.append(g_loss)

#             # If at save interval, then save generated review samples
#             if epoch % save_interval == 0:
#                 generated_review, real_review = self.saveGeneratedReview()
#                 generated_review_list.append(generated_review)
#                 real_review_list.append(real_review)

#         # Counting time elapsed
#         time_delta = time.time() - start
#         print('Training completed time:', time_delta)

#         return self.generator, self.discriminator, d_loss_list, g_loss_list, generated_review_list, real_review_list, 

In [14]:
# reviews

In [15]:
# reviews = pd.read_csv('Data/realreviews.csv', index_col = 0)

In [16]:
# reviews['label'] = 0
# reviews = remove_over(reviews)

In [17]:
# display(reviews.head())
# reviews.drop(['labels'], axis=1, inplace=True)
# reviews

In [34]:
class DetectionGAN(object):
    
    def __init__(self, dataframe, gpt_model=gpt_model, gpt_tokenizer=gpt_tokenizer, BERT_model=bert_model, BERT_tokenizer=bert_tokenizer, num_labels=2):
        
        self.dataframe = dataframe

        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
#         self.device = torch.device('cpu')

        self.bert_tokenizer = BERT_tokenizer
        self.discriminator = BERT_model.to(self.device)
        self.bert_optimizer = AdamW(self.discriminator.parameters(), lr=1e-5)

        self.gpt2_tokenizer = gpt_tokenizer
        self.generator = gpt_model.to(self.device)
        self.gpt2_optimizer = AdamW(self.generator.parameters(), lr=1e-5)
        
        torch.cuda.empty_cache()

    def text_gen(self, generator_input):
        text_id = generator_input
        inpt, past = torch.tensor([text_id]).to(self.device), None
        for _ in range(random.randint(20, 80)):
            prob, past = self.generator(inpt, past=past)
            inpt = torch.multinomial(F.softmax(prob[:,-1]), 1)
            text_id.append(inpt.item())
        return self.gpt2_tokenizer.decode(text_id)

    def prompt_gen(self, batch=2):
        sample_text = self.dataframe['reviews'].iloc[random.sample(range(len(self.dataframe)), batch)]
        first_5_text = sample_text.map(lambda x: self.gpt2_tokenizer.encode(x)[:5])
        first5_gen = first_5_text.map(self.text_gen)
        return sample_text, first5_gen

    def discrim_input(self, text):
        inpt_token = ['[CLS]'] + self.bert_tokenizer.tokenize(text) + ['[SEP]']
        inpt_id = self.bert_tokenizer.convert_tokens_to_ids(inpt_token)
        return [inpt_id]

    def save_gen_review(self):
        content = self.dataframe['reviews'].values[random.randint(0, len(self.dataframe))]
        content_id = self.gpt2_tokenizer.encode(content)[:5]
        return self.text_gen(content_id), content

    def _train(self, num_epochs = 1000, save=100):
        start = time.time()
        generated_review_list = []
        real_review_list = []
        dloss = []
        gloss = []

        for epoch in range(num_epochs):
            try:
                print('-' * 10)
                print('Epoch {}/{}'.format(epoch + 1, num_epochs))
                print('-' * 10)

                sample_text, first5_gen = self.prompt_gen(batch=1)
                dis_input = pd.concat([first5_gen,
                                       sample_text], axis = 0,
                                       ignore_index = True).map(self.discrim_input)
                dis_input = torch.LongTensor(np.array(pd.DataFrame(dis_input.sum()).fillna(0).astype('int32'))).to(self.device)
                dis_generate = dis_input[:len(range(120))].to(self.device)

                labels = torch.LongTensor([0] * len(range(120)) + [1] * len(sample_text)).to(self.device)
                valid = torch.LongTensor([1] * len(sample_text)).to(self.device)

                for phase in ['training_discriminator', 'training_generator']:
                    if phase=='training_discriminator':
                        print(phase)
                        self.discriminator.train()
                        for param in self.discriminator.parameters():
                            param.requires_grad = True

                        self.bert_optimizer.zero_grad()

                        d_loss = self.discriminator(sent_id=dis_input[:120]).mean()

                        d_loss.backward()
                        self.bert_optimizer.step()

                    else:
                        print(phase)
                        self.discriminator.eval()

                        self.gpt2_optimizer.zero_grad()

                        g_loss = self.discriminator(sent_id = dis_generate[:120]).mean()

                        g_loss.backward()
                        self.gpt2_optimizer.step() 

                print('-' * 10)
                print('Discriminator Loss:', d_loss)
                print('Generator Loss:', g_loss)
                print('-' * 10)
                dloss.append(d_loss.item())
                gloss.append(g_loss.item())

                # If at save interval, then save generated review samples
                if epoch % save == 0:
                    generated_review, real_review = self.save_gen_review()
                    generated_review_list.append(generated_review)
                    real_review_list.append(real_review)
            except:
                pass
    
            # Counting time elapsed
            time_delta = time.time() - start
            print('Training completed time:', time_delta)

        return self.generator, self.discriminator, dloss, gloss, generated_review_list, real_review_list

In [35]:
reviews = pd.read_csv('Data/realreviews.csv', index_col = 0)
display(reviews)
reviews['label'] = 0
reviews = remove_over(reviews)
display(reviews.head())
reviews.drop(['labels'], axis=1, inplace=True)
reviews

,reviewText
0,Great book.... Keeps you coming back for more....
1,Thanks!
2,Required text book for college English class. ...
3,The good thing about this book is i have got h...
4,First off Heather Dahlgren this is an amazing ...
...,...
49996,Very interesting book. With a million new book...
49997,"This was an excellent, grounded piece of liter..."
49998,"For this reader, Feverborn, the eighth of ten ..."
49999,I have been enjoying each one of the Ruth Gall...


,reviews,labels
0,Great book.... Keeps you coming back for more....,0
1,Thanks!,0
2,Required text book for college English class. ...,0
3,The good thing about this book is i have got h...,0
4,First off Heather Dahlgren this is an amazing ...,0


,reviews
0,Great book.... Keeps you coming back for more....
1,Thanks!
2,Required text book for college English class. ...
3,The good thing about this book is i have got h...
4,First off Heather Dahlgren this is an amazing ...
...,...
37459,This book gives some great insight into the wo...
37460,Great start to a PNR series. Lucien Thorne was...
37461,"This was an excellent, grounded piece of liter..."
37462,I have been enjoying each one of the Ruth Gall...


In [36]:
model = DetectionGAN(dataframe=reviews)

In [37]:
gen, dis, d_loss, g_loss, gen_reviews, real_reviews = model._train()

----------
Epoch 1/1000
----------
training_discriminator
training_generator
Training completed time: 0.9751131534576416
----------
Epoch 2/1000
----------
training_discriminator
training_generator
Training completed time: 1.4531145095825195
----------
Epoch 3/1000
----------
training_discriminator
training_generator
Training completed time: 2.3631138801574707
----------
Epoch 4/1000
----------
training_discriminator
training_generator
Training completed time: 3.018624782562256
----------
Epoch 5/1000
----------
training_discriminator
training_generator
Training completed time: 3.8606247901916504
----------
Epoch 6/1000
----------
training_discriminator
Training completed time: 5.064626932144165
----------
Epoch 7/1000
----------
training_discriminator
training_generator
Training completed time: 5.943734169006348
----------
Epoch 8/1000
----------
training_discriminator
training_generator
Training completed time: 6.508732557296753
----------
Epoch 9/1000
----------
training_discriminat

training_discriminator
training_generator
Training completed time: 53.42806625366211
----------
Epoch 71/1000
----------
training_discriminator
training_generator
Training completed time: 53.775066614151
----------
Epoch 72/1000
----------
training_discriminator
training_generator
Training completed time: 54.78906846046448
----------
Epoch 73/1000
----------
training_discriminator
training_generator
Training completed time: 55.86064124107361
----------
Epoch 74/1000
----------
training_discriminator
Training completed time: 57.1886625289917
----------
Epoch 75/1000
----------
training_discriminator
training_generator
Training completed time: 58.156672954559326
----------
Epoch 76/1000
----------
training_discriminator
training_generator
Training completed time: 59.424681663513184
----------
Epoch 77/1000
----------
training_discriminator
Training completed time: 60.7676956653595
----------
Epoch 78/1000
----------
training_discriminator
training_generator
Training completed time: 61.64

training_discriminator
training_generator
Training completed time: 110.51983761787415
----------
Epoch 140/1000
----------
training_discriminator
training_generator
Training completed time: 111.0568380355835
----------
Epoch 141/1000
----------
training_discriminator
training_generator
Training completed time: 111.99283790588379
----------
Epoch 142/1000
----------
training_discriminator
training_generator
Training completed time: 112.76483798027039
----------
Epoch 143/1000
----------
training_discriminator
training_generator
Training completed time: 113.38144874572754
----------
Epoch 144/1000
----------
training_discriminator
training_generator
Training completed time: 114.16708326339722
----------
Epoch 145/1000
----------
training_discriminator
training_generator
Training completed time: 115.17017078399658
----------
Epoch 146/1000
----------
training_discriminator
training_generator
Training completed time: 116.17320418357849
----------
Epoch 147/1000
----------
training_discrimi

Training completed time: 162.1961510181427
----------
Epoch 215/1000
----------
Training completed time: 162.56214833259583
----------
Epoch 216/1000
----------
Training completed time: 162.86414766311646
----------
Epoch 217/1000
----------
Training completed time: 163.25514721870422
----------
Epoch 218/1000
----------
Training completed time: 163.58014750480652
----------
Epoch 219/1000
----------
Training completed time: 163.89914870262146
----------
Epoch 220/1000
----------
Training completed time: 164.29715013504028
----------
Epoch 221/1000
----------
Training completed time: 164.60614824295044
----------
Epoch 222/1000
----------
Training completed time: 164.9521484375
----------
Epoch 223/1000
----------
Training completed time: 165.4181489944458
----------
Epoch 224/1000
----------
Training completed time: 165.8461492061615
----------
Epoch 225/1000
----------
Training completed time: 166.27814936637878
----------
Epoch 226/1000
----------
Training completed time: 166.673148

training_discriminator
training_generator
Training completed time: 202.1700165271759
----------
Epoch 310/1000
----------
training_discriminator
training_generator
Training completed time: 202.91601467132568
----------
Epoch 311/1000
----------
training_discriminator
training_generator
Training completed time: 203.98101544380188
----------
Epoch 312/1000
----------
training_discriminator
training_generator
Training completed time: 204.52501559257507
----------
Epoch 313/1000
----------
training_discriminator
training_generator
Training completed time: 205.04601550102234
----------
Epoch 314/1000
----------
training_discriminator
training_generator
Training completed time: 206.00601482391357
----------
Epoch 315/1000
----------
training_discriminator
training_generator
Training completed time: 206.75201654434204
----------
Epoch 316/1000
----------
training_discriminator
training_generator
Training completed time: 207.63801503181458
----------
Epoch 317/1000
----------
training_discrimi

...

Training completed time: 345.40701508522034
----------
Epoch 527/1000
----------
Training completed time: 345.64601588249207
----------
Epoch 528/1000
----------
Training completed time: 345.87001514434814
----------
Epoch 529/1000
----------
Training completed time: 346.0970149040222
----------
Epoch 530/1000
----------
Training completed time: 346.32301592826843
----------
Epoch 531/1000
----------
Training completed time: 346.55901551246643
----------
Epoch 532/1000
----------
Training completed time: 346.79001545906067
----------
Epoch 533/1000
----------
Training completed time: 347.0230162143707
----------
Epoch 534/1000
----------
Training completed time: 347.2480149269104
----------
Epoch 535/1000
----------
Training completed time: 347.4750163555145
----------
Epoch 536/1000
----------
Training completed time: 347.69901490211487
----------
Epoch 537/1000
----------
Training completed time: 347.92201566696167
----------
Epoch 538/1000
----------
Training completed time: 348.162

Training completed time: 371.57631850242615
----------
Epoch 629/1000
----------
Training completed time: 371.85431694984436
----------
Epoch 630/1000
----------
Training completed time: 372.1343173980713
----------
Epoch 631/1000
----------
Training completed time: 372.3993167877197
----------
Epoch 632/1000
----------
Training completed time: 372.6693172454834
----------
Epoch 633/1000
----------
Training completed time: 372.9383170604706
----------
Epoch 634/1000
----------
Training completed time: 373.212317943573
----------
Epoch 635/1000
----------
Training completed time: 373.4863181114197
----------
Epoch 636/1000
----------
Training completed time: 373.75831747055054
----------
Epoch 637/1000
----------
Training completed time: 374.0373182296753
----------
Epoch 638/1000
----------
Training completed time: 374.3173177242279
----------
Epoch 639/1000
----------
Training completed time: 374.597318649292
----------
Epoch 640/1000
----------
Training completed time: 374.8633172512

Training completed time: 397.3803267478943
----------
Epoch 731/1000
----------
Training completed time: 397.6293272972107
----------
Epoch 732/1000
----------
Training completed time: 397.87632727622986
----------
Epoch 733/1000
----------
Training completed time: 398.12232756614685
----------
Epoch 734/1000
----------
Training completed time: 398.3663284778595
----------
Epoch 735/1000
----------
Training completed time: 398.67932772636414
----------
Epoch 736/1000
----------
Training completed time: 398.943327665329
----------
Epoch 737/1000
----------
Training completed time: 399.2123312950134
----------
Epoch 738/1000
----------
Training completed time: 399.48532724380493
----------
Epoch 739/1000
----------
Training completed time: 399.76632857322693
----------
Epoch 740/1000
----------
Training completed time: 400.05432748794556
----------
Epoch 741/1000
----------
Training completed time: 400.35032773017883
----------
Epoch 742/1000
----------
Training completed time: 400.65332

Training completed time: 425.2543270587921
----------
Epoch 833/1000
----------
training_discriminator
Training completed time: 425.52133083343506
----------
Epoch 834/1000
----------
Training completed time: 425.7283284664154
----------
Epoch 835/1000
----------
Training completed time: 425.95332765579224
----------
Epoch 836/1000
----------
Training completed time: 426.18132853507996
----------
Epoch 837/1000
----------
Training completed time: 426.40632677078247
----------
Epoch 838/1000
----------
Training completed time: 426.64532709121704
----------
Epoch 839/1000
----------
Training completed time: 426.8703291416168
----------
Epoch 840/1000
----------
Training completed time: 427.0943295955658
----------
Epoch 841/1000
----------
Training completed time: 427.3403277397156
----------
Epoch 842/1000
----------
Training completed time: 427.5693287849426
----------
Epoch 843/1000
----------
Training completed time: 427.7953271865845
----------
Epoch 844/1000
----------
Training com

Training completed time: 451.194326877594
----------
Epoch 935/1000
----------
Training completed time: 451.4243276119232
----------
Epoch 936/1000
----------
Training completed time: 451.6533274650574
----------
Epoch 937/1000
----------
Training completed time: 451.8873269557953
----------
Epoch 938/1000
----------
Training completed time: 452.12632751464844
----------
Epoch 939/1000
----------
Training completed time: 452.3593273162842
----------
Epoch 940/1000
----------
Training completed time: 452.6053273677826
----------
Epoch 941/1000
----------
Training completed time: 452.8553273677826
----------
Epoch 942/1000
----------
Training completed time: 453.0923271179199
----------
Epoch 943/1000
----------
Training completed time: 453.3283271789551
----------
Epoch 944/1000
----------
Training completed time: 453.59332752227783
----------
Epoch 945/1000
----------
Training completed time: 453.83932876586914
----------
Epoch 946/1000
----------
Training completed time: 454.078327894

In [38]:
gen_reviews

[]

In [39]:
real_reviews

[]